# Prep

In [30]:
# Import packages
import pandas as pd
import plotly.graph_objects as go
import networkx as nx
from ds_common_utils.aux.io.snowflake_tools import SnowflakeTools
from math import log
from networkx.algorithms.community import greedy_modularity_communities
import matplotlib.pyplot as plt
import community.community_louvain as community_louvain
import numpy as np

In [31]:
# Get connection
con = SnowflakeTools().get_snowflake_ctx(
     method = 'token',
    user = '250807',
    role = 'INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD',
    warehouse = 'INSIGHT_ANALYST_WH'
)
with open('sql/item_association.sql', 'r') as query:
        df = pd.read_sql_query(query.read(), con)
with open('sql/excluded_class_list.sql', 'r') as query:
        target_item_class = pd.read_sql_query(query.read(), con)

snowflake_tools - 2025-06-19 19:09:17.999658+10:00 - Generating new token with 'INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD' role and 'SESSION:ROLE-ANY' scope...


snowflake_tools - 2025-06-19 19:09:23.507954+10:00 - Saved token with '2025-06-19 23:09:23+10:00' expiry...


In [32]:
with open('sql/overview.sql', 'r') as query:
        supplier_items = pd.read_sql_query(query.read(), con)
supplier_items        

,ITEM_NUMBER,ITEM_DESCRIPTION,ITEM_CLASS_NAME,TRANSACTIONS,SALES_SUM,QUANTITY_SUM
0,TOTAL,TOTAL,TOTAL,1106,51751093,70917
1,0823814,SKYLIGHT FIXED VELUX++780X1400MM FSM082004A,500 ROOF WINDOWS,88,3578519,5285
2,0820072,SKYLIGHT FLAT ROOF VELUX++665X1275MM FCM224600...,500 ROOF WINDOWS,68,2157456,4025
3,0823806,SKYLIGHT FIXED VELUX++780X1180MM FSM062004A,500 ROOF WINDOWS,56,2006913,3185
4,0415002,SKYLIGHT OPENABLE ELEC VELUX++780X980MM VSEM0...,500 ROOF WINDOWS,12,1740639,910
...,...,...,...,...,...,...
145,0296315,VELUX FLASHING KIT++EDWSK060000,500 SKYLIGHT ACCESSORIES,2,0,0
146,0811278,VELUX SKYLIGHT ACC++SOLAR HONEYCOMB FSCD C041045,500 SKYLIGHT ACCESSORIES,2,0,0
147,0296335,VELUX FLASHING KIT++EDLMK08 0000,500 SKYLIGHT ACCESSORIES,4,0,0
148,0296327,VELUX FLASHING KIT++EDLMK04 0000,500 SKYLIGHT ACCESSORIES,2,0,0


Accross 2 years, 2023-06-18 to 2025-06-18, 150 distinct items sold from this supplier. Total transactions 1.1k sales $52m, quantity 71k.

# Parameter tuning

In [33]:
TRX_THRESHOLD = 10 # Min N frequent pairs
QUANTILE = 0.975 # Nth percentile of metric similarity
TOP_N = 5 # top-N associated item classes per primary class
RESO = 0 # >1.0 → more, smaller communities
METRIC = 'JACCARD' # LIFT or JACCARD

3Y
TRX_THRESHOLD = 10 # Min N frequent pairs
QUANTILE = 0.9 # Nth percentile of metric similarity
TOP_N = 3 # top-N associated item classes per primary class
RESO = 0.5 # >1.0 → more, smaller communities
METRIC = 'JACCARD' # LIFT or JACCARD

# Working

In [34]:
filtered_df = df[
    ~(
        df['PRIMARY_ID'].isin(target_item_class['ITEM_CLASS_NAME']) |
        df['SECONDARY_ID'].isin(target_item_class['ITEM_CLASS_NAME'])
    )
]

filtered_df = filtered_df[filtered_df['PAIR_TRANSACTIONS'] >= TRX_THRESHOLD]
threshold = filtered_df[METRIC].quantile(QUANTILE)
filtered_df = filtered_df[filtered_df[METRIC] >= threshold]
top_edges = filtered_df.sort_values(METRIC, ascending=False).groupby('PRIMARY_ID').head(TOP_N)

In [35]:
filtered_df.sort_values(by='PAIR_TRANSACTIONS', ascending=False)

,PRIMARY_ID,SECONDARY_ID,PRIMARY_TRANSACTIONS,SECONDARY_TRANSACTIONS,TRX_START_MONTH,TRX_END_MONTH,PAIR_TRANSACTIONS,LIFT,JACCARD
3,500 ADHESIVE HOOKS REMOVABLE,500 ENAMEL PAINT,1,1,2025-05-01,2025-05-01,40,2040.0,-1.052632
4,500 ADHESIVE HOOKS REMOVABLE,500 HPM EXCL,1,1,2025-05-01,2025-05-01,40,2040.0,-1.052632
969,500 PARTICLEBOARD FLOORING,500 ADHESIVES,1,1,2025-03-01,2025-03-01,40,2120.0,-1.052632
976,500 PARTICLEBOARD FLOORING,500 METAL SCREWS,1,1,2025-03-01,2025-03-01,40,2120.0,-1.052632
979,500 PARTICLEBOARD FLOORING,500 OTHER INSULATION BATTS,1,1,2025-03-01,2025-03-01,40,2120.0,-1.052632
1478,500 STRUCT FRAMG H3 2 GR LINKD,500 FIXINGS OFF RANGE,1,1,2025-02-01,2025-02-01,40,1560.0,-1.052632
1479,500 STRUCT FRAMG H3 2 GR LINKD,500 PAINT FILLERS,1,1,2025-02-01,2025-02-01,40,1560.0,-1.052632
1488,500 STRUCT FRAMG H3 2 KD LINKD,500 GLOVES AND SPONGES,1,1,2024-10-01,2024-10-01,40,2280.0,-1.052632
1489,500 STRUCT FRAMG H3 2 KD LINKD,500 INDOOR CHEMICALS,1,1,2024-10-01,2024-10-01,40,2280.0,-1.052632
1497,500 STRUCT FRAMG H3 2 KD LINKD,501 DOOR HINGE BUTT,1,1,2024-10-01,2024-10-01,40,2280.0,-1.052632


In [36]:
mode='lines',
line_shape='spline'
    
# Build node size map from your DataFrame
node_size_map = dict(zip(top_edges['PRIMARY_ID'], top_edges['PRIMARY_TRANSACTIONS']))

# Convert values to NumPy array
sizes = np.array(list(node_size_map.values()))

# Use np.ptp instead of .ptp()
norm_sizes = (sizes - sizes.min()) / np.ptp(sizes)  # scale to 0–1
scaled_sizes = 10 + norm_sizes * 30                 # scale to 10–40

# Final size map: Product ID → Scaled Size
scaled_size_map = dict(zip(node_size_map.keys(), scaled_sizes))

# Step 1: Build the full graph
G = nx.Graph()
for _, row in top_edges.iterrows():
    G.add_edge(row['PRIMARY_ID'], row['SECONDARY_ID'], weight=row[METRIC])

# Step 2: Louvain community detection
partition = community_louvain.best_partition(G, weight=METRIC, resolution=RESO)

# Step 3: Layout
pos = nx.spring_layout(G, weight='weight', seed=42
                      )

# Step 4: Edge trace (no filter on weight)
edge_x, edge_y = [], []
for u, v in G.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    edge_x += [x0, x1, None]
    edge_y += [y0, y1, None]

edge_trace = go.Scatter(
    x=edge_x,
    y=edge_y,
    line=dict(width=1, color='gray'),
    hoverinfo='none',
    mode='lines'
)

# Step 5: Node trace with Louvain cluster coloring
color_list = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
              '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
              '#bcbd22', '#17becf']  # Extend if you expect >10 clusters

node_x, node_y, node_color, node_size, hover_text = [], [], [], [], []

for node in G.nodes():
    x, y = pos[node]
    cluster = partition[node]
    size = scaled_size_map.get(node, 10)  # default to small if missing

    node_x.append(x)
    node_y.append(y)
    node_color.append(color_list[cluster % len(color_list)])
    node_size.append(size)
    hover_text.append(f"Product ID: {node}<br>Cluster: {cluster}<br>Transactions: {int(size)}")

node_trace = go.Scatter(
    x=node_x,
    y=node_y,
    mode='markers+text',
    hoverinfo='text',
    text=[str(node) for node in G.nodes()],
    textposition='top center',
    marker=dict(
        color=node_color,
        size=node_size,  # <--- size now dynamic!
        line=dict(width=2, color='black')
    ),
    textfont=dict(
    size=16,          # increase for visibility
    color='black',
    family='Arial'
)
)
# Step 6: Plot
fig = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        width=1200,          # increase width
    height=800, 
        title=dict(text='Louvain Product Clustering (All Edges)', x=0.5, font=dict(size=16)),
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=20, r=20, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
    )
)

/tmp/ipykernel_3822/4149448309.py:11: RuntimeWarning:

invalid value encountered in divide



ValueError: cannot convert float NaN to integer

# Results

In [ ]:
fig.show()

In [ ]:
partition_df = pd.DataFrame.from_dict(partition, orient='index', columns=['segment']).reset_index()
partition_df.rename(columns={'index': 'PRIMARY_ID'}, inplace=True)
labelled_df = filtered_df.merge(partition_df, on='PRIMARY_ID', how='left')
labelled_df = labelled_df[['PRIMARY_ID','PRIMARY_TRANSACTIONS', 'segment']].drop_duplicates()
labelled_df.sort_values(by='segment')

,PRIMARY_ID,PRIMARY_TRANSACTIONS,segment
0,500 ROOF WINDOWS,772,0
2,500 SKYLIGHT ACCESSORIES,484,1


New skylight framing or structural retrofit:
Segment 0 – Roof cutout prep and reinforcement

Roof recladding or cut-in during roof works:
Segment 1 – Mixed Site Prep, Sheathing, and Noise

Full-frame integration for high-quality install:
Segment 2 – Fibre Cement Integration & Segment 3 – Connector Framing & Timber

Internal ceiling refit or thermal efficiency upgrade:
Segment 4 – Insulation & Finishing Support

In [ ]:
labelled_df.to_csv('Labelled_class_segment.csv',index=False)